In [6]:
import cv2
import numpy as np

In [ ]:
def shadow_remove(img):
    rgb_planes = cv2.split(img)
    result_norm_planes = []
    for plane in rgb_planes:
        dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
        bg_img = cv2.medianBlur(dilated_img, 21)
        diff_img = 255 - cv2.absdiff(plane, bg_img)
        norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        result_norm_planes.append(norm_img)
    shadowremov = cv2.merge(result_norm_planes)
    return shadowremov

In [7]:
def hog_preprocessing(image):
    resized_image = cv2.resize(image,(500,500))
    shad_image = shadow_remove(resized_image)
    grayscale_image = cv2.cvtColor(shad_image, cv2.COLOR_BGR2GRAY)
    filtered_image  = cv2.GaussianBlur(grayscale_image, (3, 3), 1.0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    normalized_image = clahe.apply(filtered_image)
    img_array = np.array(normalized_image, dtype=np.float32) / 255.0
    corrected_array = np.power(img_array, 0.5)
    corrected_image = np.uint8(corrected_array * 255.0)
    resized_image = cv2.resize(corrected_image,(128,128))
    return resized_image